# Machine Learning Engineer Nanodegree
# Capstone
# Project: To find an Optimum Machine Learning Model for Human Activity        Recognition with Smartphone Sensor Data

This is the Capstone for MLND.
Our purpose is to recognize human activity based on the sensor data from smartphones and to find an optimum model with can predict these activities with high degree of accuracy.

In [1]:
# Import the Necessary Libraries

import time, pickle
import numpy as np
from numpy import array
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, log_loss

# Use pickle to open stored objects.
with open('data.pickle','rb') as f:
    data = pickle.load(f)
with open('X_train.pickle','rb') as f:
    X_train = pickle.load(f)
with open('X_test.pickle','rb') as f:
    X_test = pickle.load(f)
with open('y_train.pickle','rb') as f:
    y_train = pickle.load(f)
with open('y_test.pickle','rb') as f:
    y_test = pickle.load(f)
with open('y_true.pickle','rb') as f:
    y_true = pickle.load(f)

# Keras Neural Network

In [2]:
# Import required modules( keras model as sequential. Dense, Dropout and Activation Layers and SGD as the optimizer)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


### Data Prperation for the Keras Model:

Lets first check our dataset and convert it into a format that Karas accepts.

Currently We have our data as X_train, X_test, Y_train and y_test with all numerical values as dataframe.

It should be noted that Keras only takes input data in Numpy Array format. So we have to convert our dataset to a numpy array. As our datset is 2-dimensional, we can either use 'numpy.asarray()' or use '.as_matrix' for our conversion.

In [3]:
# Convert the train and test features to arrays
train_samples = np.asarray(X_train)
test_samples = np.array(X_test)

# Encode y_train using One Hot Encoder (y_test is already converted as y_true) 
onehot_encoder = OneHotEncoder(sparse = False)
integer_encoded = y_train
integer_encoded = integer_encoded.reshape(len(y_train), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
y_train_ohe = onehot_encoded

In [4]:
# Define a function to predict scores
def check_score(y_val, x_val):
    pred = model.predict_classes(x_val)
    preds = model.predict_proba(x_val)
    f_score = f1_score(y_val,pred, average = 'weighted')
    y_n = pd.DataFrame(y_val)
    y_n = pd.get_dummies(y_n)
    lgls = log_loss(np.array(y_n),preds)
    runtime = (end - start)
    print('F1_score for Keras Sequential Model is : ',str(f_score))
    print('Logloss Score for Keras Sequential Model is is : ',str(lgls))
    print('Runtime is: ', str(runtime))


### Base Model Architecture

First, we will use the below architecture for our sequential model. Further we will add more layers to our model and change its parameters to try and obtain optimum results.

In [16]:
# Sequential model for KerasClassifier
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (561, )))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# Compile model
optimizer = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
# Add checkpoint
check = ModelCheckpoint('best_weight_basic.h5', monitor='val_acc', verbose=1, 
                        save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 64)                35968     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 390       
Total params: 40,518
Trainable params: 40,518
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Fit the model to the Train Data with epochs as 10 and batch size 64. We are keeping 10% data for validation set.
start = time.clock()
model.fit(train_samples, y_train_ohe, epochs=20, callbacks = [check],
          validation_split= 0.10, shuffle = True, batch_size=64)
end = time.clock()

Train on 6488 samples, validate on 721 samples
Epoch 1/20
6488/6488 [==============================] - 0s - loss: 1.3052 - acc: 0.4094 - val_loss: 0.7336 - val_acc: 0.7282
Epoch 2/20
6488/6488 [==============================] - 0s - loss: 0.8294 - acc: 0.6191 - val_loss: 0.5079 - val_acc: 0.7656
Epoch 3/20
6488/6488 [==============================] - 0s - loss: 0.6657 - acc: 0.7039 - val_loss: 0.3867 - val_acc: 0.8571
Epoch 4/20
6488/6488 [==============================] - 0s - loss: 0.5760 - acc: 0.7424 - val_loss: 0.3466 - val_acc: 0.8863
Epoch 5/20
6488/6488 [==============================] - 0s - loss: 0.5135 - acc: 0.7710 - val_loss: 0.3022 - val_acc: 0.9029
Epoch 6/20
6488/6488 [==============================] - 0s - loss: 0.4846 - acc: 0.7871 - val_loss: 0.2700 - val_acc: 0.9265
Epoch 7/20
6488/6488 [==============================] - 0s - loss: 0.4520 - acc: 0.8050 - val_loss: 0.2398 - val_acc: 0.9154
Epoch 8/20
6488/6488 [==============================] - 0s - loss: 0.4217 - ac

In [18]:
# Load best model weights
model.load_weights('best_weight_basic.h5')

# Predict score of the basic model
check_score(y_test,test_samples)

2432/3090 [======================>.......] - ETA: 0s('F1_score for Keras Sequential Model is : ', '0.957234772889')
('Logloss Score for Keras Sequential Model is is : ', '0.15301331721')
('Runtime is: ', '20.393846')


In [19]:
# Clear session
from keras import backend as k
k.clear_session

<function keras.backend.tensorflow_backend.clear_session>

## Let us add more layers to our model and check how it performs against the base model.

In [20]:
# Improve model by adding more layers
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (561,)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
# Compile model
optimizer = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
check = ModelCheckpoint('best_weights.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 64)                35968     
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               131584    
__________

In [21]:
model.fit(train_samples, y_train_ohe, epochs = 20, validation_split= 0.10, shuffle = True, callbacks = [check], batch_size = 64)

Train on 6488 samples, validate on 721 samples
Epoch 1/20
6488/6488 [==============================] - 0s - loss: 1.2247 - acc: 0.4101 - val_loss: 0.6705 - val_acc: 0.6865
Epoch 2/20
6488/6488 [==============================] - 0s - loss: 0.6499 - acc: 0.7005 - val_loss: 0.3695 - val_acc: 0.8114
Epoch 3/20
6488/6488 [==============================] - 0s - loss: 0.3802 - acc: 0.8365 - val_loss: 0.2061 - val_acc: 0.9182
Epoch 4/20
6488/6488 [==============================] - 0s - loss: 0.3042 - acc: 0.8762 - val_loss: 0.2259 - val_acc: 0.8946
Epoch 5/20
6488/6488 [==============================] - 0s - loss: 0.2552 - acc: 0.8980 - val_loss: 0.1825 - val_acc: 0.9376
Epoch 6/20
6488/6488 [==============================] - 0s - loss: 0.2419 - acc: 0.9023 - val_loss: 0.1497 - val_acc: 0.9487
Epoch 7/20
6488/6488 [==============================] - 0s - loss: 0.2158 - acc: 0.9091 - val_loss: 0.1476 - val_acc: 0.9417
Epoch 8/20
6488/6488 [==============================] - 0s - loss: 0.1972 - ac

In [22]:
model.load_weights('best_weights.h5')
check_score(y_test,test_samples)

2432/3090 [======================>.......] - ETA: 0s('F1_score for Keras Sequential Model is : ', '0.954998008324')
('Logloss Score for Keras Sequential Model is is : ', '0.109943819876')
('Runtime is: ', '20.393846')


In [23]:
# Clear session
from keras import backend as k
k.clear_session

<function keras.backend.tensorflow_backend.clear_session>

** _We can see that our score detoriated from the base model. Let us change no of epochs and train again as we have added more layer._**

In [24]:
# Improve model by adding more layers
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (561,)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
# Compile model
optimizer = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
check = ModelCheckpoint('best_weights.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 64)                35968     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               131584    
__________

In [25]:
start = time.clock()
model.fit(train_samples, y_train_ohe, epochs = 40, validation_split= 0.10, 
          shuffle = True, callbacks = [check], batch_size = 64)
end = time.clock()

Train on 6488 samples, validate on 721 samples
Epoch 1/40
6488/6488 [==============================] - 1s - loss: 1.2169 - acc: 0.4111 - val_loss: 0.6970 - val_acc: 0.7323
Epoch 2/40
6488/6488 [==============================] - 0s - loss: 0.6441 - acc: 0.7007 - val_loss: 0.3481 - val_acc: 0.8682
Epoch 3/40
6488/6488 [==============================] - 0s - loss: 0.3991 - acc: 0.8298 - val_loss: 0.3203 - val_acc: 0.8447
Epoch 4/40
6488/6488 [==============================] - 0s - loss: 0.2951 - acc: 0.8761 - val_loss: 0.3144 - val_acc: 0.8502
Epoch 5/40
6488/6488 [==============================] - 0s - loss: 0.2457 - acc: 0.9006 - val_loss: 0.1611 - val_acc: 0.9362
Epoch 6/40
6488/6488 [==============================] - 0s - loss: 0.2306 - acc: 0.9040 - val_loss: 0.2192 - val_acc: 0.8821
Epoch 7/40
6488/6488 [==============================] - 0s - loss: 0.2189 - acc: 0.9072 - val_loss: 0.1817 - val_acc: 0.9251
Epoch 8/40
6488/6488 [==============================] - 0s - loss: 0.2037 - ac

6488/6488 [==============================] - 0s - loss: 0.1737 - acc: 0.9400 - val_loss: 0.1223 - val_acc: 0.9570
Epoch 33/40
6488/6488 [==============================] - 0s - loss: 0.1313 - acc: 0.9545 - val_loss: 0.0729 - val_acc: 0.9736
Epoch 34/40
6488/6488 [==============================] - 0s - loss: 0.1415 - acc: 0.9473 - val_loss: 0.0833 - val_acc: 0.9736
Epoch 35/40
6488/6488 [==============================] - 0s - loss: 0.1416 - acc: 0.9505 - val_loss: 0.1021 - val_acc: 0.9626
Epoch 36/40
6488/6488 [==============================] - 0s - loss: 0.1471 - acc: 0.9470 - val_loss: 0.0864 - val_acc: 0.9723
Epoch 37/40
6488/6488 [==============================] - 0s - loss: 0.1113 - acc: 0.9619 - val_loss: 0.0656 - val_acc: 0.9723
Epoch 38/40
6488/6488 [==============================] - 0s - loss: 0.1405 - acc: 0.9527 - val_loss: 0.0819 - val_acc: 0.9709
Epoch 39/40
6488/6488 [==============================] - 0s - loss: 0.1268 - acc: 0.9542 - val_loss: 0.0770 - val_acc: 0.9764
Epoc

In [26]:
model.load_weights('best_weights.h5')
check_score(y_test,test_samples)

3040/3090 [============================>.] - ETA: 0s('F1_score for Keras Sequential Model is : ', '0.974100913773')
('Logloss Score for Keras Sequential Model is is : ', '0.0833727516097')
('Runtime is: ', '20.393846')


* As we can see, this time our model performance has improved a lot. Lets try to imrpove it further by changing other parameters.

In [32]:
# Clear session
from keras import backend as k
k.clear_session

<function keras.backend.tensorflow_backend.clear_session>

In [34]:
# Improve model by changing optimizer as adam.
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (561, )))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
# Compile model
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
check = ModelCheckpoint('best_weights.h5', monitor='val_acc', verbose=1, 
                        save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 64)                35968     
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 512)               131584    
__________

In [35]:
start = time.clock()
model.fit(train_samples, y_train_ohe, epochs = 100, validation_split= 0.10, 
          shuffle = True, callbacks = [check], batch_size = 64)
end = time.clock()

Train on 6488 samples, validate on 721 samples
Epoch 1/100
6488/6488 [==============================] - 1s - loss: 0.9222 - acc: 0.5717 - val_loss: 0.3914 - val_acc: 0.8419
Epoch 2/100
6488/6488 [==============================] - 0s - loss: 0.3774 - acc: 0.8382 - val_loss: 0.2456 - val_acc: 0.9057
Epoch 3/100
6488/6488 [==============================] - 0s - loss: 0.2670 - acc: 0.8935 - val_loss: 0.1812 - val_acc: 0.9320
Epoch 4/100
6488/6488 [==============================] - 0s - loss: 0.2192 - acc: 0.9155 - val_loss: 0.1443 - val_acc: 0.9431
Epoch 5/100
6488/6488 [==============================] - 1s - loss: 0.1679 - acc: 0.9351 - val_loss: 0.1195 - val_acc: 0.9556
Epoch 6/100
6488/6488 [==============================] - 0s - loss: 0.1754 - acc: 0.9348 - val_loss: 0.1372 - val_acc: 0.9487
Epoch 7/100
6488/6488 [==============================] - 0s - loss: 0.1736 - acc: 0.9350 - val_loss: 0.1157 - val_acc: 0.9570
Epoch 8/100
6488/6488 [==============================] - 0s - loss: 0.1

6488/6488 [==============================] - 0s - loss: 0.0992 - acc: 0.9613 - val_loss: 0.0996 - val_acc: 0.9723
Epoch 32/100
6488/6488 [==============================] - 0s - loss: 0.1005 - acc: 0.9605 - val_loss: 0.0982 - val_acc: 0.9709
Epoch 33/100
6488/6488 [==============================] - 0s - loss: 0.1053 - acc: 0.9588 - val_loss: 0.1018 - val_acc: 0.9695
Epoch 34/100
6488/6488 [==============================] - 1s - loss: 0.0935 - acc: 0.9625 - val_loss: 0.0954 - val_acc: 0.9736
Epoch 35/100
6488/6488 [==============================] - 1s - loss: 0.1141 - acc: 0.9588 - val_loss: 0.1156 - val_acc: 0.9792
Epoch 36/100
6488/6488 [==============================] - 1s - loss: 0.1215 - acc: 0.9564 - val_loss: 0.1023 - val_acc: 0.9750
Epoch 37/100
6488/6488 [==============================] - 1s - loss: 0.1100 - acc: 0.9598 - val_loss: 0.1013 - val_acc: 0.9709
Epoch 38/100
6488/6488 [==============================] - 0s - loss: 0.1119 - acc: 0.9581 - val_loss: 0.0983 - val_acc: 0.97

6488/6488 [==============================] - 1s - loss: 0.0898 - acc: 0.9706 - val_loss: 0.1001 - val_acc: 0.9792
Epoch 65/100
6488/6488 [==============================] - 0s - loss: 0.1168 - acc: 0.9641 - val_loss: 0.0886 - val_acc: 0.9778
Epoch 66/100
6488/6488 [==============================] - 0s - loss: 0.0878 - acc: 0.9721 - val_loss: 0.0909 - val_acc: 0.9820
Epoch 67/100
6488/6488 [==============================] - 1s - loss: 0.0987 - acc: 0.9689 - val_loss: 0.0753 - val_acc: 0.9792
Epoch 68/100
6488/6488 [==============================] - 1s - loss: 0.1096 - acc: 0.9638 - val_loss: 0.0974 - val_acc: 0.9653
Epoch 69/100
6488/6488 [==============================] - 1s - loss: 0.0956 - acc: 0.9692 - val_loss: 0.1033 - val_acc: 0.9736
Epoch 70/100
6488/6488 [==============================] - 1s - loss: 0.0870 - acc: 0.9718 - val_loss: 0.0649 - val_acc: 0.9834
Epoch 71/100
6488/6488 [==============================] - 0s - loss: 0.1089 - acc: 0.9673 - val_loss: 0.0612 - val_acc: 0.97

6488/6488 [==============================] - 0s - loss: 0.0976 - acc: 0.9687 - val_loss: 0.0716 - val_acc: 0.9806
Epoch 98/100
6488/6488 [==============================] - 0s - loss: 0.0922 - acc: 0.9736 - val_loss: 0.0838 - val_acc: 0.9778
Epoch 99/100
6488/6488 [==============================] - 0s - loss: 0.0998 - acc: 0.9678 - val_loss: 0.0777 - val_acc: 0.9778
Epoch 100/100
6488/6488 [==============================] - 1s - loss: 0.0924 - acc: 0.9718 - val_loss: 0.0782 - val_acc: 0.9778


In [36]:
model.load_weights('best_weights.h5')
check_score(y_test,test_samples)

3008/3090 [============================>.] - ETA: 0s('F1_score for Keras Sequential Model is : ', '0.979906259617')
('Logloss Score for Keras Sequential Model is is : ', '0.0709449485182')
('Runtime is: ', '20.393846')


* With optimizer as Adam our accuracy improved. Now lets change the no of epochs, learning rate and batch size to improve the score further.

In [38]:
# Clear session
from keras import backend as k
k.clear_session

<function keras.backend.tensorflow_backend.clear_session>

In [39]:
# Tune Parameters: by increasing batch size and epochs
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (561, )))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
# Compile model
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
check = ModelCheckpoint('best_weights.h5', monitor='val_acc', verbose=10, 
                        save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 64)                35968     
_________________________________________________________________
dropout_37 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_38 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_39 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 512)               131584    
__________

In [40]:
start = time.clock()
model.fit(train_samples, y_train_ohe, epochs = 200, validation_split= 0.10, 
          shuffle = True, callbacks = [check], batch_size = 128)
end = time.clock()

Train on 6488 samples, validate on 721 samples
Epoch 1/200
6488/6488 [==============================] - 0s - loss: 1.2287 - acc: 0.4191 - val_loss: 0.5299 - val_acc: 0.7836
Epoch 2/200
6488/6488 [==============================] - 0s - loss: 0.4847 - acc: 0.7875 - val_loss: 0.2574 - val_acc: 0.9126
Epoch 3/200
6488/6488 [==============================] - 0s - loss: 0.3075 - acc: 0.8735 - val_loss: 0.1846 - val_acc: 0.9279
Epoch 4/200
6488/6488 [==============================] - 0s - loss: 0.2362 - acc: 0.9088 - val_loss: 0.1550 - val_acc: 0.9445
Epoch 5/200
6488/6488 [==============================] - 0s - loss: 0.2004 - acc: 0.9225 - val_loss: 0.1336 - val_acc: 0.9528
Epoch 6/200
6488/6488 [==============================] - 0s - loss: 0.1720 - acc: 0.9367 - val_loss: 0.1302 - val_acc: 0.9598
Epoch 7/200
6488/6488 [==============================] - 0s - loss: 0.1588 - acc: 0.9391 - val_loss: 0.1294 - val_acc: 0.9515
Epoch 8/200
6488/6488 [==============================] - 0s - loss: 0.1

6488/6488 [==============================] - 0s - loss: 0.0766 - acc: 0.9696 - val_loss: 0.0771 - val_acc: 0.9736
Epoch 33/200
6488/6488 [==============================] - 0s - loss: 0.0768 - acc: 0.9703 - val_loss: 0.0732 - val_acc: 0.9792
Epoch 34/200
6488/6488 [==============================] - 0s - loss: 0.0713 - acc: 0.9741 - val_loss: 0.1011 - val_acc: 0.9709
Epoch 35/200
6488/6488 [==============================] - 0s - loss: 0.0800 - acc: 0.9703 - val_loss: 0.0711 - val_acc: 0.9820
Epoch 36/200
6488/6488 [==============================] - 0s - loss: 0.0777 - acc: 0.9706 - val_loss: 0.0890 - val_acc: 0.9764
Epoch 37/200
6488/6488 [==============================] - 0s - loss: 0.0773 - acc: 0.9709 - val_loss: 0.0824 - val_acc: 0.9778
Epoch 38/200
6488/6488 [==============================] - 0s - loss: 0.0833 - acc: 0.9687 - val_loss: 0.0900 - val_acc: 0.9764
Epoch 39/200
6488/6488 [==============================] - 0s - loss: 0.0841 - acc: 0.9676 - val_loss: 0.0887 - val_acc: 0.96

6488/6488 [==============================] - 0s - loss: 0.1019 - acc: 0.9647 - val_loss: 0.1008 - val_acc: 0.9612
Epoch 66/200
6488/6488 [==============================] - 0s - loss: 0.0788 - acc: 0.9709 - val_loss: 0.0778 - val_acc: 0.9709
Epoch 67/200
6488/6488 [==============================] - 0s - loss: 0.0654 - acc: 0.9749 - val_loss: 0.1003 - val_acc: 0.9723
Epoch 68/200
6488/6488 [==============================] - 0s - loss: 0.0736 - acc: 0.9732 - val_loss: 0.0811 - val_acc: 0.9806
Epoch 69/200
6488/6488 [==============================] - 0s - loss: 0.0555 - acc: 0.9795 - val_loss: 0.0866 - val_acc: 0.9778
Epoch 70/200
6488/6488 [==============================] - 0s - loss: 0.0717 - acc: 0.9756 - val_loss: 0.0802 - val_acc: 0.9764
Epoch 71/200
6488/6488 [==============================] - 0s - loss: 0.0722 - acc: 0.9766 - val_loss: 0.0892 - val_acc: 0.9723
Epoch 72/200
6488/6488 [==============================] - 0s - loss: 0.0758 - acc: 0.9729 - val_loss: 0.0851 - val_acc: 0.97

6488/6488 [==============================] - 0s - loss: 0.0583 - acc: 0.9798 - val_loss: 0.0823 - val_acc: 0.9792
Epoch 99/200
6488/6488 [==============================] - 0s - loss: 0.0705 - acc: 0.9733 - val_loss: 0.0928 - val_acc: 0.9723
Epoch 100/200
6488/6488 [==============================] - 0s - loss: 0.0616 - acc: 0.9780 - val_loss: 0.0709 - val_acc: 0.9778
Epoch 101/200
6488/6488 [==============================] - 0s - loss: 0.0664 - acc: 0.9747 - val_loss: 0.1061 - val_acc: 0.9626
Epoch 102/200
6488/6488 [==============================] - 0s - loss: 0.0617 - acc: 0.9787 - val_loss: 0.0938 - val_acc: 0.9736
Epoch 103/200
6488/6488 [==============================] - 0s - loss: 0.0597 - acc: 0.9770 - val_loss: 0.0716 - val_acc: 0.9820
Epoch 104/200
6488/6488 [==============================] - 0s - loss: 0.0583 - acc: 0.9784 - val_loss: 0.0842 - val_acc: 0.9709
Epoch 105/200
6488/6488 [==============================] - 0s - loss: 0.0562 - acc: 0.9810 - val_loss: 0.0702 - val_acc

6488/6488 [==============================] - 0s - loss: 0.0850 - acc: 0.9676 - val_loss: 0.1680 - val_acc: 0.9223
Epoch 132/200
6488/6488 [==============================] - 0s - loss: 0.0767 - acc: 0.9729 - val_loss: 0.1031 - val_acc: 0.9709
Epoch 133/200
6488/6488 [==============================] - 0s - loss: 0.0795 - acc: 0.9698 - val_loss: 0.1235 - val_acc: 0.9445
Epoch 134/200
6488/6488 [==============================] - 0s - loss: 0.0685 - acc: 0.9764 - val_loss: 0.1224 - val_acc: 0.9542
Epoch 135/200
6488/6488 [==============================] - 0s - loss: 0.0692 - acc: 0.9758 - val_loss: 0.1252 - val_acc: 0.9431
Epoch 136/200
6488/6488 [==============================] - 0s - loss: 0.0761 - acc: 0.9706 - val_loss: 0.1059 - val_acc: 0.9709
Epoch 137/200
6488/6488 [==============================] - 0s - loss: 0.0696 - acc: 0.9727 - val_loss: 0.1581 - val_acc: 0.9404
Epoch 138/200
6488/6488 [==============================] - 0s - loss: 0.0639 - acc: 0.9772 - val_loss: 0.1144 - val_ac

6488/6488 [==============================] - 0s - loss: 0.0696 - acc: 0.9726 - val_loss: 0.0668 - val_acc: 0.9806
Epoch 165/200
6488/6488 [==============================] - 0s - loss: 0.0589 - acc: 0.9761 - val_loss: 0.0900 - val_acc: 0.9792
Epoch 166/200
6488/6488 [==============================] - 0s - loss: 0.0713 - acc: 0.9730 - val_loss: 0.0921 - val_acc: 0.9695
Epoch 167/200
6488/6488 [==============================] - 0s - loss: 0.0575 - acc: 0.9784 - val_loss: 0.1130 - val_acc: 0.9653
Epoch 168/200
6488/6488 [==============================] - 0s - loss: 0.0683 - acc: 0.9752 - val_loss: 0.1315 - val_acc: 0.9681
Epoch 169/200
6488/6488 [==============================] - 0s - loss: 0.0503 - acc: 0.9809 - val_loss: 0.0935 - val_acc: 0.9639
Epoch 170/200
6488/6488 [==============================] - 0s - loss: 0.0604 - acc: 0.9770 - val_loss: 0.0992 - val_acc: 0.9570
Epoch 171/200
6488/6488 [==============================] - 0s - loss: 0.0457 - acc: 0.9830 - val_loss: 0.0843 - val_ac

6488/6488 [==============================] - 0s - loss: 0.0770 - acc: 0.9723 - val_loss: 0.0969 - val_acc: 0.9653
Epoch 198/200
6488/6488 [==============================] - 0s - loss: 0.0657 - acc: 0.9764 - val_loss: 0.1113 - val_acc: 0.9528
Epoch 199/200
6488/6488 [==============================] - 0s - loss: 0.0549 - acc: 0.9795 - val_loss: 0.1052 - val_acc: 0.9750
Epoch 200/200
6488/6488 [==============================] - 0s - loss: 0.0600 - acc: 0.9760 - val_loss: 0.0838 - val_acc: 0.9778


In [41]:
model.load_weights('best_weights.h5')
check_score(y_test,test_samples)

2880/3090 [==========================>...] - ETA: 0s('F1_score for Keras Sequential Model is : ', '0.981831446682')
('Logloss Score for Keras Sequential Model is is : ', '0.0726247957959')
('Runtime is: ', '158.621221')


In [42]:
# Clear session
from keras import backend as k
k.clear_session

<function keras.backend.tensorflow_backend.clear_session>

In [46]:
# Now lets decrease the learning rate while increasing epochs
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (561, )))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
# Compile model
optimizer = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
check = ModelCheckpoint('best_weights.h5', monitor='val_acc', verbose=10, 
                        save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 64)                35968     
_________________________________________________________________
dropout_45 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_58 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_46 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_47 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 512)               131584    
__________

In [47]:
start = time.clock()
model.fit(train_samples, y_train_ohe, epochs = 500, validation_split= 0.10, 
          shuffle = True, callbacks = [check], batch_size = 128)
end = time.clock()

Train on 6488 samples, validate on 721 samples
Epoch 1/500
6488/6488 [==============================] - 0s - loss: 1.3638 - acc: 0.3591 - val_loss: 0.8398 - val_acc: 0.6283
Epoch 2/500
6488/6488 [==============================] - 0s - loss: 0.7452 - acc: 0.6588 - val_loss: 0.4471 - val_acc: 0.8017
Epoch 3/500
6488/6488 [==============================] - 0s - loss: 0.4364 - acc: 0.8147 - val_loss: 0.2611 - val_acc: 0.8988
Epoch 4/500
6488/6488 [==============================] - 0s - loss: 0.2953 - acc: 0.8809 - val_loss: 0.2017 - val_acc: 0.9196
Epoch 5/500
6488/6488 [==============================] - 0s - loss: 0.2326 - acc: 0.9137 - val_loss: 0.1620 - val_acc: 0.9362
Epoch 6/500
6488/6488 [==============================] - 0s - loss: 0.2160 - acc: 0.9162 - val_loss: 0.1541 - val_acc: 0.9487
Epoch 7/500
6488/6488 [==============================] - 0s - loss: 0.1817 - acc: 0.9297 - val_loss: 0.1271 - val_acc: 0.9528
Epoch 8/500
6488/6488 [==============================] - 0s - loss: 0.1

6488/6488 [==============================] - 0s - loss: 0.0688 - acc: 0.9767 - val_loss: 0.0840 - val_acc: 0.9750
Epoch 32/500
6488/6488 [==============================] - 0s - loss: 0.0786 - acc: 0.9716 - val_loss: 0.0724 - val_acc: 0.9736
Epoch 33/500
6488/6488 [==============================] - 0s - loss: 0.0644 - acc: 0.9773 - val_loss: 0.0721 - val_acc: 0.9792
Epoch 34/500
6488/6488 [==============================] - 0s - loss: 0.0586 - acc: 0.9783 - val_loss: 0.0737 - val_acc: 0.9792
Epoch 35/500
6488/6488 [==============================] - 0s - loss: 0.0663 - acc: 0.9764 - val_loss: 0.1286 - val_acc: 0.9626
Epoch 36/500
6488/6488 [==============================] - 0s - loss: 0.0644 - acc: 0.9756 - val_loss: 0.0690 - val_acc: 0.9764
Epoch 37/500
6488/6488 [==============================] - 0s - loss: 0.0665 - acc: 0.9753 - val_loss: 0.1015 - val_acc: 0.9653
Epoch 38/500
6488/6488 [==============================] - 0s - loss: 0.0628 - acc: 0.9761 - val_loss: 0.0823 - val_acc: 0.97

6488/6488 [==============================] - 0s - loss: 0.0577 - acc: 0.9792 - val_loss: 0.1209 - val_acc: 0.9695
Epoch 65/500
6488/6488 [==============================] - 0s - loss: 0.0827 - acc: 0.9666 - val_loss: 0.0847 - val_acc: 0.9792
Epoch 66/500
6488/6488 [==============================] - 0s - loss: 0.0524 - acc: 0.9804 - val_loss: 0.0705 - val_acc: 0.9820
Epoch 67/500
6488/6488 [==============================] - 0s - loss: 0.0572 - acc: 0.9810 - val_loss: 0.0775 - val_acc: 0.9764
Epoch 68/500
6488/6488 [==============================] - 0s - loss: 0.0432 - acc: 0.9830 - val_loss: 0.0756 - val_acc: 0.9736
Epoch 69/500
6488/6488 [==============================] - 0s - loss: 0.0520 - acc: 0.9804 - val_loss: 0.0762 - val_acc: 0.9820
Epoch 70/500
6488/6488 [==============================] - 0s - loss: 0.0445 - acc: 0.9841 - val_loss: 0.0894 - val_acc: 0.9792
Epoch 71/500
6488/6488 [==============================] - 0s - loss: 0.0554 - acc: 0.9803 - val_loss: 0.0924 - val_acc: 0.97

6488/6488 [==============================] - 0s - loss: 0.0565 - acc: 0.9784 - val_loss: 0.0876 - val_acc: 0.9750
Epoch 98/500
6488/6488 [==============================] - 0s - loss: 0.0501 - acc: 0.9835 - val_loss: 0.0790 - val_acc: 0.9750
Epoch 99/500
6488/6488 [==============================] - 0s - loss: 0.0688 - acc: 0.9741 - val_loss: 0.0803 - val_acc: 0.9820
Epoch 100/500
6488/6488 [==============================] - 0s - loss: 0.0448 - acc: 0.9821 - val_loss: 0.0859 - val_acc: 0.9709
Epoch 101/500
6488/6488 [==============================] - 0s - loss: 0.0450 - acc: 0.9846 - val_loss: 0.0840 - val_acc: 0.9764
Epoch 102/500
6488/6488 [==============================] - 0s - loss: 0.0406 - acc: 0.9847 - val_loss: 0.0696 - val_acc: 0.9778
Epoch 103/500
6488/6488 [==============================] - 0s - loss: 0.0453 - acc: 0.9840 - val_loss: 0.0703 - val_acc: 0.9778
Epoch 104/500
6488/6488 [==============================] - 0s - loss: 0.0481 - acc: 0.9817 - val_loss: 0.0711 - val_acc:

6488/6488 [==============================] - 0s - loss: 0.0393 - acc: 0.9860 - val_loss: 0.0902 - val_acc: 0.9778
Epoch 131/500
6488/6488 [==============================] - 0s - loss: 0.0697 - acc: 0.9743 - val_loss: 0.1186 - val_acc: 0.9723
Epoch 132/500
6488/6488 [==============================] - 0s - loss: 0.0540 - acc: 0.9804 - val_loss: 0.0716 - val_acc: 0.9792
Epoch 133/500
6488/6488 [==============================] - 0s - loss: 0.0497 - acc: 0.9830 - val_loss: 0.0784 - val_acc: 0.9778
Epoch 134/500
6488/6488 [==============================] - 0s - loss: 0.0457 - acc: 0.9823 - val_loss: 0.0952 - val_acc: 0.9695
Epoch 135/500
6488/6488 [==============================] - 0s - loss: 0.0598 - acc: 0.9752 - val_loss: 0.0650 - val_acc: 0.9820
Epoch 136/500
6488/6488 [==============================] - 0s - loss: 0.0524 - acc: 0.9803 - val_loss: 0.0798 - val_acc: 0.9764
Epoch 137/500
6488/6488 [==============================] - 0s - loss: 0.0513 - acc: 0.9812 - val_loss: 0.0469 - val_ac

...

6488/6488 [==============================] - 0s - loss: 0.0342 - acc: 0.9855 - val_loss: 0.0828 - val_acc: 0.9778
Epoch 362/500
6488/6488 [==============================] - 0s - loss: 0.0285 - acc: 0.9900 - val_loss: 0.0677 - val_acc: 0.9847
Epoch 363/500
6488/6488 [==============================] - 0s - loss: 0.0285 - acc: 0.9901 - val_loss: 0.0720 - val_acc: 0.9806
Epoch 364/500
6488/6488 [==============================] - 0s - loss: 0.0195 - acc: 0.9946 - val_loss: 0.0821 - val_acc: 0.9820
Epoch 365/500
6488/6488 [==============================] - 0s - loss: 0.0262 - acc: 0.9918 - val_loss: 0.1217 - val_acc: 0.9695
Epoch 366/500
6488/6488 [==============================] - 0s - loss: 0.0392 - acc: 0.9869 - val_loss: 0.0839 - val_acc: 0.9834
Epoch 367/500
6488/6488 [==============================] - 0s - loss: 0.0271 - acc: 0.9892 - val_loss: 0.0897 - val_acc: 0.9820
Epoch 368/500
6488/6488 [==============================] - 0s - loss: 0.0379 - acc: 0.9838 - val_loss: 0.0895 - val_ac

6488/6488 [==============================] - 0s - loss: 0.0568 - acc: 0.9801 - val_loss: 0.0719 - val_acc: 0.9764
Epoch 395/500
6488/6488 [==============================] - 0s - loss: 0.0442 - acc: 0.9814 - val_loss: 0.0734 - val_acc: 0.9847
Epoch 396/500
6488/6488 [==============================] - 0s - loss: 0.0359 - acc: 0.9857 - val_loss: 0.0675 - val_acc: 0.9847
Epoch 397/500
6488/6488 [==============================] - 0s - loss: 0.0347 - acc: 0.9877 - val_loss: 0.0758 - val_acc: 0.9806
Epoch 398/500
6488/6488 [==============================] - 0s - loss: 0.0334 - acc: 0.9887 - val_loss: 0.0867 - val_acc: 0.9847
Epoch 399/500
6488/6488 [==============================] - 0s - loss: 0.0266 - acc: 0.9889 - val_loss: 0.0875 - val_acc: 0.9792
Epoch 400/500
6488/6488 [==============================] - 0s - loss: 0.0287 - acc: 0.9900 - val_loss: 0.0985 - val_acc: 0.9778
Epoch 401/500
6488/6488 [==============================] - 0s - loss: 0.0268 - acc: 0.9904 - val_loss: 0.0771 - val_ac

6488/6488 [==============================] - 0s - loss: 0.0200 - acc: 0.9920 - val_loss: 0.0799 - val_acc: 0.9820
Epoch 428/500
6488/6488 [==============================] - 0s - loss: 0.0201 - acc: 0.9923 - val_loss: 0.0823 - val_acc: 0.9834
Epoch 429/500
6488/6488 [==============================] - 0s - loss: 0.0239 - acc: 0.9909 - val_loss: 0.0837 - val_acc: 0.9820
Epoch 430/500
6488/6488 [==============================] - 0s - loss: 0.0234 - acc: 0.9911 - val_loss: 0.0845 - val_acc: 0.9764
Epoch 431/500
6488/6488 [==============================] - 0s - loss: 0.0217 - acc: 0.9921 - val_loss: 0.0714 - val_acc: 0.9820
Epoch 432/500
6488/6488 [==============================] - 0s - loss: 0.0192 - acc: 0.9937 - val_loss: 0.0866 - val_acc: 0.9792
Epoch 433/500
6488/6488 [==============================] - 0s - loss: 0.0239 - acc: 0.9914 - val_loss: 0.0769 - val_acc: 0.9834
Epoch 434/500
6488/6488 [==============================] - 0s - loss: 0.0249 - acc: 0.9901 - val_loss: 0.0861 - val_ac

6488/6488 [==============================] - 0s - loss: 0.0484 - acc: 0.9841 - val_loss: 0.0720 - val_acc: 0.9792
Epoch 461/500
6488/6488 [==============================] - 0s - loss: 0.0339 - acc: 0.9881 - val_loss: 0.0736 - val_acc: 0.9778
Epoch 462/500
6488/6488 [==============================] - 0s - loss: 0.0267 - acc: 0.9897 - val_loss: 0.0764 - val_acc: 0.9792
Epoch 463/500
6488/6488 [==============================] - 0s - loss: 0.0285 - acc: 0.9908 - val_loss: 0.0711 - val_acc: 0.9778
Epoch 464/500
6488/6488 [==============================] - 0s - loss: 0.0255 - acc: 0.9917 - val_loss: 0.0880 - val_acc: 0.9820
Epoch 465/500
6488/6488 [==============================] - 0s - loss: 0.0398 - acc: 0.9858 - val_loss: 0.1126 - val_acc: 0.9750
Epoch 466/500
6488/6488 [==============================] - 0s - loss: 0.0274 - acc: 0.9909 - val_loss: 0.0792 - val_acc: 0.9806
Epoch 467/500
6488/6488 [==============================] - 0s - loss: 0.0316 - acc: 0.9883 - val_loss: 0.0795 - val_ac

6488/6488 [==============================] - 0s - loss: 0.0269 - acc: 0.9904 - val_loss: 0.0932 - val_acc: 0.9806
Epoch 494/500
6488/6488 [==============================] - 0s - loss: 0.0424 - acc: 0.9844 - val_loss: 0.0561 - val_acc: 0.9834
Epoch 495/500
6488/6488 [==============================] - 0s - loss: 0.0240 - acc: 0.9912 - val_loss: 0.0727 - val_acc: 0.9806
Epoch 496/500
6488/6488 [==============================] - 0s - loss: 0.0252 - acc: 0.9908 - val_loss: 0.0801 - val_acc: 0.9792
Epoch 497/500
6488/6488 [==============================] - 0s - loss: 0.0346 - acc: 0.9861 - val_loss: 0.0886 - val_acc: 0.9792
Epoch 498/500
6488/6488 [==============================] - 0s - loss: 0.0482 - acc: 0.9795 - val_loss: 0.1050 - val_acc: 0.9695
Epoch 499/500
6488/6488 [==============================] - 0s - loss: 0.0276 - acc: 0.9901 - val_loss: 0.0815 - val_acc: 0.9792
Epoch 500/500
6488/6488 [==============================] - 0s - loss: 0.0226 - acc: 0.9918 - val_loss: 0.0695 - val_ac

In [48]:
model.load_weights('best_weights.h5')
check_score(y_test,test_samples)

2976/3090 [===========================>..] - ETA: 0s('F1_score for Keras Sequential Model is : ', '0.982137574772')
('Logloss Score for Keras Sequential Model is is : ', '0.0742955556088')
('Runtime is: ', '374.608259')


### Results: Below are the results for our optimum Keras NN

* F1_score for Keras Sequential Model is : 0.982137574772
* Logloss Score for Keras Sequential Model is: 0.0742955556088
* Runtime is: 374.608259 Seconds